# Part 2: Mining the web: Web APIs

We hope the preceding exercise was painful: even with tools to process HTML, it is rough downloading raw HTML and trying to extract information from it!

> Can you think of any other reasons why scraping websites for data in this way is not a good idea?

Luckily, many websites provide an application programming interface (API) for querying their data or otherwise accessing their services from your programs. For instance, Twitter provides a web API for gathering tweets, Flickr provides one for gathering image data, and Github for accessing information about repository histories.

These kinds of web APIs are much easier to use than, for instance, the preceding technique which scrapes raw web pages and then has to parse the resulting HTML. Moreover, there are more scalable in the sense that the web servers can transmit structured data in a less verbose form than raw HTML.

As a starting example, here is some code to look at the activity on Github related to the public version of our course's materials.

In [8]:
import requests

response = requests.get ('https://api.github.com/repos/cse6040/labs-fa17/events')

print ("==> .headers:", response.headers, "\n")

==> .headers: {'Date': 'Mon, 01 Oct 2018 06:12:49 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Server': 'GitHub.com', 'Status': '200 OK', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '48', 'X-RateLimit-Reset': '1538377834', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding', 'ETag': 'W/"9aa2a636e423429ef8cc14ce06471d43"', 'Last-Modified': 'Sun, 30 Sep 2018 15:21:05 GMT', 'X-Poll-Interval': '60', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Access-Control-Expose-Headers': 'ETag, Link, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Referrer-Policy': 'origin-when-cross-origin, strict-origi

Note the `Content-Type` of the response:

In [9]:
print (response.headers['Content-Type'])

application/json; charset=utf-8


The response is in JSON format, which is an open format for exchanging semi-structured data. (JSON stands for **J**ava**S**cript **O**bject **N**otation.) JSON is designed to be human-readable and machine-readable, and maps especially well in Python to nested dictionaries. Let's take a look.

> See also [this tutorial](https://www.w3schools.com/js/js_json_intro.asp) for a JSON primer. JSON is among _the_ universal formats for sharing data on the web; see, for instance, https://www.sitepoint.com/10-example-json-files/.

In [10]:
import json
print(type(response.json ()))
print(json.dumps(response.json()[:3], sort_keys=True, indent=2))

<class 'list'>
[
  {
    "actor": {
      "avatar_url": "https://avatars.githubusercontent.com/u/8300000?",
      "display_login": "cizara",
      "gravatar_id": "",
      "id": 8300000,
      "login": "cizara",
      "url": "https://api.github.com/users/cizara"
    },
    "created_at": "2018-09-30T15:21:05Z",
    "id": "8344319562",
    "org": {
      "avatar_url": "https://avatars.githubusercontent.com/u/31073927?",
      "gravatar_id": "",
      "id": 31073927,
      "login": "cse6040",
      "url": "https://api.github.com/orgs/cse6040"
    },
    "payload": {
      "action": "started"
    },
    "public": true,
    "repo": {
      "id": 100506580,
      "name": "cse6040/labs-fa17",
      "url": "https://api.github.com/repos/cse6040/labs-fa17"
    },
    "type": "WatchEvent"
  },
  {
    "actor": {
      "avatar_url": "https://avatars.githubusercontent.com/u/2495855?",
      "display_login": "govind-mukundan",
      "gravatar_id": "",
      "id": 2495855,
      "login": "govind-muku

**Exercise 0.** It should be self-evident that the JSON response above consists of a sequence of records, which we will refer to as _events_. Each event is associated with an _actor_. Write some code to extract a dictionary of all actors, where the key is the actor's login and the value is the actor's URL.

In [11]:
def extract_actors (json_github_events):
    """Given JSON records for events in a GitHub repo,
    returns a dictionary of the actors and their URLs.
    """
    urls = {}
    for event in json_github_events:
        actor = event['actor']['display_login']
        url = event['actor']['url']
        urls[actor] = url
    return urls

In [12]:
actor_urls = extract_actors(response.json ())

for actor, url in actor_urls.items ():
    print ('{}: {}'.format(actor, url))
    assert url == "https://api.github.com/users/{}".format(actor)

cizara: https://api.github.com/users/cizara
govind-mukundan: https://api.github.com/users/govind-mukundan
chanafee: https://api.github.com/users/chanafee
chenkang001: https://api.github.com/users/chenkang001
cookblaze: https://api.github.com/users/cookblaze
dwrussell: https://api.github.com/users/dwrussell
fractalbass: https://api.github.com/users/fractalbass
msa444: https://api.github.com/users/msa444
tonyelhabr: https://api.github.com/users/tonyelhabr
zili-chen: https://api.github.com/users/zili-chen


**Exercise 1.** Write some code that goes to each actor's URL and determines their name. If an actor URL is invalid, that actor should not appear in the output.

In [13]:
def lookup_names (actor_urls):
    """Given a dictionary of (actor, url) pairs, looks up the JSON at
    the URL and extracts the user's name (if any). Returns a new
    dictionary of (actor, name) pairs.
    """
    import re
    
    names = {}
    for actor, url in actor_urls.items ():
        response = requests.get (url)
        
        # Possible error conditions
        if response is None: continue
        if re.search ('application/json', response.headers['Content-Type']) is None: continue
        if 'name' not in response.json (): continue
            
        names[actor] = response.json ()['name']
    return names

In [14]:
actor_names = lookup_names (actor_urls)

for actor, name in actor_names.items ():
    print ("{}: {}".format (actor, name))
    
assert actor_names['bryanlie'] == 'Dan Li'

cizara: Lucho
govind-mukundan: eGovind
chanafee: None
chenkang001: None
cookblaze: None
dwrussell: None
fractalbass: Miles Porter
msa444: None
tonyelhabr: Tony ElHabr
zili-chen: Zili Chen


KeyError: 'bryanlie'

That's the end of this notebook. Processing JSON is fairly straightforward, because it maps very naturally to nested dictionaries in Python. You might search the web for other sources of JSON data, including [this one](https://www.yelp.com/dataset/challenge), and do your own processing!